In [2]:
import time
import pandas as pd
from census import Census
import altair as alt
import numpy as np

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

In [40]:
# Rent Distribution: Table B25056 total housing units renting paying cash
rent_variables = {
    "B25056_002E":"total",
    "B25056_002M":"total_moe",
    "B25056_003E":"less_100",
    "B25056_003M":"less_100_moe",
    "B25056_004E":"100-149",
    "B25056_004M":"100-149_moe",
    "B25056_005E":"150-199",
    "B25056_005M":"150-199_moe",
    "B25056_006E":"200-249",
    "B25056_006M":"200-249_moe",
    "B25056_007E":"250-299",
    "B25056_007M":"250-299_moe",
    "B25056_008E":"300-349",
    "B25056_008M":"300-349_moe",
    "B25056_009E":"350-399",
    "B25056_009M":"350-399_moe",
    "B25056_010E":"400-449",
    "B25056_010M":"400-449_moe",
    "B25056_011E":"450-499",
    "B25056_011M":"450-499_moe",
    "B25056_012E":"500-549",
    "B25056_012M":"500-549_moe",
    "B25056_013E":"550-599",
    "B25056_013M":"550-599_moe",
    "B25056_014E":"600-649",
    "B25056_014M":"600-649_moe",
    "B25056_015E":"650-699",
    "B25056_015M":"650-699_moe",
    "B25056_016E":"700-749",
    "B25056_016M":"700-749_moe",
    "B25056_017E":"750-799",
    "B25056_017M":"750-799_moe",
    "B25056_018E":"800-899",
    "B25056_018M":"800-899_moe",
    "B25056_019E":"900-999",
    "B25056_019M":"900-999_moe",
    "B25056_020E":"1000-1249",
    "B25056_020M":"1000-1249_moe",
    "B25056_021E":"1250-1499",
    "B25056_021M":"1250-1499_moe",
    "B25056_022E":"1500-1999",
    "B25056_022M":"1500-1999_moe",
    "B25056_023E":"2000-2499",
    "B25056_023M":"2000-2499_moe",
    "B25056_024E":"2500-2999",
    "B25056_024M":"2500-2999_moe"
}
rent_variables_top = {
    "B25056_025E":"3000-3499",
    "B25056_025M":"3000-3499_moe",
    "B25056_026E":"3500_more",
    "B25056_026M":"3500_more_moe"
}
lowest_rent_cols = [
    "less_100", "100-149", "150-199", "200-249", "250-299", "300-349", "350-399", "400-449"
]
mid_low_rent_cols = [
    "500-549", "550-599", "600-649", "650-699", "700-749", "750-799", "800-899", "900-999"
]
mid_high_rent_cols = [
    "1000-1249", "1250-1499", "1500-1999", "2000-2499"
]
highest_rent_cols = [
    "2500-2999"
    #, "3000-3499", "3500_more"
]
rent_cols_out = [
    'total', 'total_moe',
    'pct_499_less', 'pct_499_less_moe',
    'pct_500-999', 'pct_500-999_moe',
    'pct_1000-2499', 'pct_1000-2499_moe',
    'pct_2500_more', 'pct_2500_more_moe'
]

In [28]:
def combine_rents(in_df):
    '''
    Outputs: simplified vehicle breakdown with MOEs
    '''
    df = in_df.copy()
    
    ### AGGREGATE ESTIMATES
    df['499_less'] = df[lowest_rent_cols].sum(axis='columns')
    # Use a list comprehension to append "_moe" to all strings in our list
    lowest_rent_moes = [f'{col}_moe' for col in lowest_rent_cols]    
    df['499_less_moe'] = (df[lowest_rent_moes]**2).sum(axis='columns')**0.5

    df['500-999'] = df[mid_low_rent_cols].sum(axis='columns')
    # Use a list comprehension to append "_moe" to all strings in our list
    mid_low_rent_moes = [f'{col}_moe' for col in mid_low_rent_cols]    
    df['500-999_moe'] = (df[mid_low_rent_moes]**2).sum(axis='columns')**0.5

    df['1000-2499'] = df[mid_high_rent_cols].sum(axis='columns')
    # Use a list comprehension to append "_moe" to all strings in our list
    mid_high_rent_moes = [f'{col}_moe' for col in mid_high_rent_cols]    
    df['1000-2499_moe'] = (df[mid_high_rent_moes]**2).sum(axis='columns')**0.5

    df['2500_more'] = df[highest_rent_cols].sum(axis='columns')
    # Use a list comprehension to append "_moe" to all strings in our list
    highest_rent_moes = [f'{col}_moe' for col in highest_rent_cols]    
    df['2500_more_moe'] = (df[highest_rent_moes]**2).sum(axis='columns')**0.5
    
    for group in ['499_less', '500-999', '1000-2499', '2500_more']:
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

        #NaN-out any too-low absolute n
        df.loc[df['total'] < 25, f'pct_{group}'] = float('NaN')
        df.loc[df['total'] < 25, f'pct_{group}_moe'] = float('NaN')
        
        #NaN-out any too-low moe
        df[f'pct_{group}_moe_ratio'] = df[f'pct_{group}_moe']/df[f'pct_{group}']
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}'] = float('NaN')
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}_moe'] = float('NaN')
    
    return df

In [36]:
def combine_tracts(in_df):
    '''

    Outputs:
    A modified version of in_df with census tracts combined
    '''
    
    # Start by making a copy of in_df, so we don't destroy the original data.
    # df is also a nice short name we can use throughout this function
    df_copy = in_df.copy()
    df = pd.DataFrame(columns=rent_variables.values())

    ### CLEAN UNUSUAL MOES
    df_copy = df_copy.replace(-555555555.0, 0)

    for c in rent_variables.values():
        if "moe" not in c:
            # sum the totals
            df.at[0, c] = df_copy[c].sum()
        else:
            # sum of squares the moes
            df.at[0, c] = np.sqrt((df_copy[c]**2).sum())

    df_out = combine_rents(df)
    
    return df_out

In [26]:
def get_rent_precombo(year_in, place_num):
    # for getting mode for census tracts before they are combined
    df = pd.DataFrame(
        c.acs5.get(
            list(rent_variables.keys()),
            {'for': place_num, 'in': 'state:06 county:013'},
            year=year_in
        )
    )
    # todo: get this later
    # df2 = pd.DataFrame(
    #     c.acs5.get(
    #         list(rent_variables_top.keys()),
    #         {'for': place_num, 'in': 'state:06 county:013'},
    #         year=year_in
    #     )
    # )
    # df = df2.join(df3)
    df = df.rename(columns=rent_variables)
    #df = df.drop(columns=["state", "county", "tract"])
    
    return df

In [6]:
def get_city_df(year_in):
    # City
    df = pd.DataFrame(
        c.acs5.get(
            list(veh_variables.keys()),
            {'for': 'place:60620', 'in': 'state:06'},
            year=year_in
        )
    )
    df = df.rename(columns=veh_variables)
    df_out = combine_vehicles(df)

    return df_out

In [27]:
# for 5yr ACS 2014 and 2019
# Get ACS Table for select tracts around the BART station
# county:013
# city: 60620
c_tracts = 'tract:375000, 376000, 377000, 374000, 381000'

df_tracts_2014 = get_rent_precombo(2014, c_tracts)

In [41]:
df_comb_2014 = combine_tracts(df_tracts_2014)
df_tracts_out_2014 = df_comb_2014[rent_cols_out]
df_tracts_out_2014.insert(0, "year", 2014)
# df_comb_2019 = combine_tracts(df_tracts_2019)
# df_tracts_out_2019 = df_comb_2019[veh_columns_out]
# df_tracts_out_2019.insert(0, "year", 2019)

In [42]:
df_tracts_out_2014

,year,total,total_moe,pct_499_less,pct_499_less_moe,pct_500-999,pct_500-999_moe,pct_1000-2499,pct_1000-2499_moe,pct_2500_more,pct_2500_more_moe
0,2014,5297.0,359.14621,0.185577,0.040685,0.376817,0.054274,0.431376,0.057322,NaN,NaN


In [9]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08141 in Richmond City
df_city_2014 = get_city_df(2014)
df_city_2019 = get_city_df(2019)

In [10]:
df_city_out_2014 = df_city_2014[veh_columns_out]
df_city_out_2014.insert(0, "year", 2014)
df_city_out_2019 = df_city_2019[veh_columns_out]
df_city_out_2019.insert(0, "year", 2019)

In [11]:
#export settings, for a combined csv
df_tracts_out = pd.concat([df_tracts_out_2014, df_tracts_out_2019])
df_city_out = pd.concat([df_city_out_2014, df_city_out_2019])

In [12]:
df_tracts_out

,year,total,total_moe,pct_no_vehicle,pct_no_vehicle_moe,pct_one_vehicle,pct_one_vehicle_moe,pct_two_or_more,pct_two_or_more_moe
0,2014,11022.0,716.115214,0.061423,0.023519,0.273272,0.036698,0.665306,0.065788
0,2019,12942.0,677.685768,NaN,NaN,0.224463,0.033649,0.747643,0.063172


In [13]:
df_city_out

,year,total,total_moe,pct_no_vehicle,pct_no_vehicle_moe,pct_one_vehicle,pct_one_vehicle_moe,pct_two_or_more,pct_two_or_more_moe
0,2014,46973.0,1272.0,0.04196,0.008740,0.244821,0.018180,0.713218,0.032811
0,2019,52366.0,1212.0,0.03340,0.009114,0.237807,0.018146,0.728793,0.035268
